<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/index_relation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import ast

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_price(self, start:int='20180501', interval='day'):
    url ='https://api.finance.naver.com/siseJson.naver?symbol=' + self.code + ' &requestType=1&startTime=' \
    + start + '&endTime=' + str(datetime.datetime.now().date().strftime("%Y%m%d")) +  '&timeframe=' + interval

    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    b = bs_obj.get_text()
    for i in ['\n', '\t', "\\", ' ']:
      b = b.replace(i,'')

    data = np.array(ast.literal_eval(b)[1:])

    dic = {'Date':list(map(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'), data[:,0])), \
      'Open':np.array(data[:,1], int), 'High':np.array(data[:,2], int), 'Low':np.array(data[:,3], int),\
       'Close':np.array(data[:,4], int), 'Volume':np.array(data[:,5], int)}

    df = pd.DataFrame(data=dic)
    df = df.set_index('Date')
    return df
    
  def get_fundamental(self):
    url = 'https://finance.naver.com/item/main.nhn?code=' + self.code
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    ths = bs_obj.find_all("th", {'scope':'col'})
    ths = [th.get_text() for th in ths][10:-22]
    dates = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),ths))
    dates = list(map(lambda x: x + '(Y)', dates[:4])) + list(map(lambda x: x + '(M)', dates[4:]))

    tr = bs_obj.find_all("tbody")
    tds = tr[2].find_all('td')
    tds = [td.get_text() for td in tds]
    elements = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),tds))
    elements = list(map(lambda x: float(x) if x.replace('.','').isdigit() else np.nan, elements))

    temp_dict = {}
    cnt = 0
    index = ['sales', 'operating profit', 'net income', 'operating margin', 'net margin', 'roe', 'debt ratio', 'quick ratio', \
    'reserve ratio', 'eps', 'per', 'bps', 'pbr', 'dividend per share', 'dividend yield ratio', 'dividend payout ratio']

    for i in dates:
      temp_dict[i] = elements[cnt::10]
      cnt += 1

    df = pd.DataFrame.from_dict(temp_dict)
    df = df.set_index([pd.Index(index)])
    return df

In [2]:
stock = Stock('KOSPI')
df = stock.get_price('20170101', 'day')
df['Change'] = df['Close']/df['Open']
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2017-01-02,202223,203179,201568,202616,229874,1.001943
2017-01-03,203431,204407,202847,204397,268127,1.004749
2017-01-04,204629,204629,204061,204564,371488,0.999682
2017-01-05,204552,204650,203949,204195,541343,0.998255
2017-01-06,204811,205184,204566,204912,455350,1.000493
...,...,...,...,...,...,...
2021-07-08,328768,329571,325241,325268,1516034,0.989354
2021-07-09,324552,324552,318880,321795,1308364,0.991505
2021-07-12,322751,325145,322751,324647,1135042,1.005874


In [5]:
# 전날 조건
pre_lower = 0
pre_upper = 1
range = df[df['Change'].between(pre_lower, pre_upper)]

# show after range
range_next = df[df.shift(1)['Change'].between(pre_lower, pre_upper)]

# 전날 조건 비례 오늘
lower = 1
upper = 2
sum(range_next['Change'].between(lower, upper)/ len(range_next['Change']))

0.5097001763668427

In [6]:
'''
달 +
주 -
날 - 
'''

'\n달 +\n주 -\n날 - \n'